#  **Antigranular** Heart Disease Prediction Contest (ft. **Harvard/OpenDP** and **TPDP**)


First of all, I would like to thank **Antigranular** for organizing this competition. It was fun participating in it, and it was my first time joining a competition of this type. I have worked on a very similar project involving the use of GANs to generate synthetic data that preserves the features of the original data. This synthetic data can be used to train deep learning models while maintaining a balance between data characteristics and privacy.

I didn't try many different approaches since my solution is straightforward. The key is to understand the `eps` and `target_delta` parameters. The idea is to use the largest values for these parameters while ensuring that the model produces good results in the first 3-5 runs. This approach is quite risky, so the model was designed to **overfit** the training data by using a significant number of hidden layers and other techniques. To minimize overfitting, I applied regularization techniques such as a high `dropout rate (40%)` and `batch normalization` layers, `batch_size=128`. Additionally, I used a small learning rate and trained the model for a large number of epochs to achieve the best results.




### 📦 Install Antigranular

This command installs the [Antigranular PyPI Package](https://pypi.org/project/antigranular/) on the local enviroment.


In [ ]:
# Install the Antigranular package
!pip install antigranular &> /dev/null

### ✍ Login to the Enclave

Head over to [Competitions](https://www.antigranular.com/competitions) to find your `<user_id>`, `<user_secret>` and the competition's name and copy that command here.

![img](https://docs.antigranular.com/shots/comp_cell.png)

In [ ]:
import antigranular as ag
session = ag.login(<client_id>,<client_secret>, competition = "Heart Disease Prediction Hackathon")

Dataset "Heart Disease Prediction Hackathon Dataset" loaded to the kernel as heart_disease_prediction_hackathon_dataset
Key Name                       Value Type     
---------------------------------------------
train_y                        PrivateDataFrame
train_x                        PrivateDataFrame
test_x                         DataFrame      

Connected to Antigranular server session id: 62253581-17db-46c2-98b2-98ba9377ac26, the session will time out if idle for 25 minutes
Cell magic '%%ag' registered successfully, use `%%ag` in a notebook cell to execute your python code on Antigranular private python server
🚀 Everything's set up and ready to roll!


### 🤖 Using AG

You can now simply use ``%%ag`` to run code on an enclave! You can always head over to our [Docs](https://docs.antigranular.com/) to learn more about AG, but for now, we can define train and test variables as follows.

In [ ]:
%%ag
x_train = heart_disease_prediction_hackathon_dataset["train_x"]
y_train = heart_disease_prediction_hackathon_dataset["train_y"]
x_test = heart_disease_prediction_hackathon_dataset["test_x"]

### Model

### Model Architecture

The model defined is a sequential neural network designed for binary classification. It is composed of the following layers:

1. **Input Layer**:
   - **Dense Layer**: 256 units, ReLU activation, input shape of 6 (which matches the number of input features).
   - **Dropout Layer**: 40% dropout rate to prevent overfitting.

2. **Hidden Layers**:
   - **Dense Layer**: 128 units, ReLU activation.
   - **Dropout Layer**: 40% dropout rate.
   - **Dense Layer**: 128 units, ReLU activation.
   - **Dropout Layer**: 40% dropout rate.
   - **Dense Layer**: 64 units, ReLU activation.
   - **Dropout Layer**: 40% dropout rate.
   - **Dense Layer**: 32 units, ReLU activation.
   - **Dropout Layer**: 40% dropout rate.
   - **Dense Layer**: 16 units, ReLU activation.
   - **Dropout Layer**: 40% dropout rate.

3. **Output Layer**:
   - **Dense Layer**: 1 unit, Sigmoid activation for binary classification output.

### Parameters and Additional Configurations

1. **Differential Privacy**:
   - **PrivateKerasModel**: The model is wrapped in a `PrivateKerasModel` to provide differential privacy guarantees.
   - **l2_norm_clip**: Set to 1, which controls the clipping of gradients during training.
   - **noise_multiplier**: Set to 1, which scales the noise added to the gradients to ensure privacy.

2. **Optimizer**:
   - **Adam Optimizer**: Used for training with a learning rate of 0.001.

3. **Loss Function and Metrics**:
   - **Loss Function**: Binary Cross-Entropy, suitable for binary classification tasks.
   - **Metrics**: Accuracy, to monitor the performance of the model during training and evaluation.

This model leverages dropout layers extensively to mitigate overfitting, while the dense layers with ReLU activation function provide the non-linear transformations needed to capture complex patterns in the data.

In [ ]:
%%ag
import tensorflow as tf
from op_pandas import standard_scaler, PrivateDataFrame
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from op_tensorflow import PrivateKerasModel, PrivateDataLoader


seqM = Sequential([
    Dense(256, activation='relu', input_shape=(6,)),
    Dropout(0.4),
    Dense(128, activation='relu'),
    Dropout(0.4),
    Dense(128, activation='relu'),
    Dropout(0.4),
    Dense(64, activation='relu'),
    Dropout(0.4),
    Dense(32, activation='relu'),
    Dropout(0.4),
    Dense(16, activation='relu'),
    Dropout(0.4),
    Dense(1, activation='sigmoid')
])

# Create DP keras model
dp_model = PrivateKerasModel(model=seqM, l2_norm_clip=1, noise_multiplier=1)

# Use a standard (non-DP) optimizer directly from keras.
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

# PrivateKerasModel uses similar API as standard Keras
dp_model.compile(
	optimizer = optimizer,
	loss = 'binary_crossentropy',
	metrics = ["accuracy"]
)

/usr/local/lib/python3.10/site-packages/tensorflow_probability/python/__init__.py:57: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if (distutils.version.LooseVersion(tf.__version__) <
/usr/local/lib/python3.10/site-packages/tensorflow_probability/python/__init__.py:58: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  distutils.version.LooseVersion(required_tensorflow_version)):



### I applied stander scaler with `eps=0.9` .

In [ ]:
%%ag
x_train_scaled = standard_scaler(x_train, eps=.9)
x_train_scaled.info()

+----+----------+-------------+---------------+---------+------------------------------------------+
|    | Column   | numerical   | categorical   | dtype   | bounds                                   |
|----+----------+-------------+---------------+---------+------------------------------------------|
|  0 | age      | True        | False         | float64 | (-3.4603125701650077, 3.192940281529251) |
|  1 | sex      | True        | False         | float64 | (-1.4680622112406603,                    |
|    |          |             |               |         | 0.7104900522995734)                      |
|  2 | bp       | True        | False         | float64 | (-3.250938543691753, 5.340360673477323)  |
|  3 | ch       | True        | False         | float64 | (-2.685157094365248, 6.491414183378572)  |
|  4 | bs       | True        | False         | float64 | (-2.0090091779602903,                    |
|    |          |             |               |         | 2.9715776116526684)              

### I used a `batch_size=128` for better generalization

In [ ]:
%%ag
data_loader = PrivateDataLoader(feature_df=x_train_scaled , label_df=y_train, batch_size=128)

### I trained the model for 50 epochs with `target_delta=1e-2`

In [ ]:
%%ag
dp_model.fit(x=data_loader, epochs=50, target_delta=1e-2)

Epoch 1/50

63/63 - 9s - loss: 0.6750 - accuracy: 0.5663 - 9s/epoch - 138ms/step

Epoch 2/50

63/63 - 2s - loss: 0.6007 - accuracy: 0.6860 - 2s/epoch - 37ms/step

Epoch 3/50

63/63 - 2s - loss: 0.5638 - accuracy: 0.7161 - 2s/epoch - 33ms/step

Epoch 4/50

63/63 - 2s - loss: 0.5430 - accuracy: 0.7302 - 2s/epoch - 36ms/step

Epoch 5/50

63/63 - 2s - loss: 0.5165 - accuracy: 0.7449 - 2s/epoch - 35ms/step

Epoch 6/50

63/63 - 2s - loss: 0.4986 - accuracy: 0.7558 - 2s/epoch - 33ms/step

Epoch 7/50

63/63 - 2s - loss: 0.4760 - accuracy: 0.7683 - 2s/epoch - 33ms/step

Epoch 8/50

63/63 - 2s - loss: 0.4548 - accuracy: 0.7877 - 2s/epoch - 33ms/step

Epoch 9/50

63/63 - 2s - loss: 0.4456 - accuracy: 0.7886 - 2s/epoch - 35ms/step

Epoch 10/50

63/63 - 2s - loss: 0.4274 - accuracy: 0.8000 - 2s/epoch - 32ms/step

Epoch 11/50

63/63 - 2s - loss: 0.4067 - accuracy: 0.8051 - 2s/epoch - 32ms/step

Epoch 12/50

63/63 - 2s - loss: 0.3927 - accuracy: 0.8163 - 2s/epoch - 32ms/step

Epoch 13/50

63/63 - 2s 

### I applied standard scaler with `eps=0.9` .  

In [ ]:
%%ag
x_test_scaler = standard_scaler(PrivateDataFrame(x_test), eps=.9)
y_pred = dp_model.predict(x_test_scaler, label_columns=["output"])

63/63 [==============================] - 1s 8ms/step



### Converting the output to 0 or 1

In [ ]:
%%ag
# Note that the predictions are a float scalar
# so we scale it
def f(x: float) -> float:
  if x > 0.5:
    return 1
  else:
    return 0

y_pred["output"] = y_pred["output"].map(f, output_bounds=(0, 1))

###  Make your submission

In [ ]:
%%ag
result = submit_predictions(y_pred)

score: {'leaderboard': 0.8967691064769541, 'logs': {'BIN_ACC': 0.9197691064769541, 'LIN_EPS': -0.022999999999999975}}

